# GSEA for CORUM Protein Complexes using Network Centrality

**Objective:** This notebook performs a Gene Set Enrichment Analysis (GSEA) to identify protein complexes that are significantly enriched for genes with high or low network centrality.

**Workflow:**
* Setup: Install and import necessary Python libraries.
* Prepare Ranked Gene List (.rnk): We will use a pre-calculated file of genes ranked by their betweenness centrality. We will format this file for GSEApy.
* Prepare Gene Set Database (.gmt): We will download the CORUM protein complex database and convert it into the required .gmt format.
* Run GSEA Pre-Ranked: We will execute the analysis using the gseapy library.
* Interpret Results: We will discuss how to understand the GSEA output plots and tables.

# Setup Environment

In [ ]:
!pip install gseapy

# Import Libraries

In [ ]:
import pandas as pd
import gseapy as gp
import os

print("Libraries imported successfully.")

# Create and Format the .rnk File

In [ ]:
# --- Configuration ---
# *** MODIFY THIS VARIABLE to point to your uploaded file ***
centrality_input_file = "/content/corum_dataset.ungraph.betweenness.GSEA.rnk"
rnk_output_file = "/content/corum_betweenness.rnk"

# --- Processing ---
try:
    # Read the data, assuming it might have a header
    df = pd.read_csv(centrality_input_file, sep='\t')

    # Ensure the columns are correctly named (adjust if necessary)
    df.columns = ['gene', 'score']

    # Sort by score in descending order (optional, but good practice)
    df = df.sort_values(by='score', ascending=False)

    # Save to a new file without the header and index
    df.to_csv(rnk_output_file, sep='\t', header=False, index=False)

    print(f"Successfully created GSEA-ready ranked list: {rnk_output_file}")
    print("\nFirst 5 lines of the .rnk file:")
    !head -n 5 {rnk_output_file}

except FileNotFoundError:
    print(f"ERROR: File not found at '{centrality_input_file}'. Please upload your file and check the name.")

# Create .gmt File from CORUM Database

In [ ]:
# This cell defines and runs a function to convert the official CORUM database
# file into the .gmt format required by GSEA. It specifically extracts human complexes.
#
# *** THIS VERSION INCLUDES A FIX to remove empty gene names. ***

# --- Configuration ---
corum_input_file = "/content/corum_allComplexes.txt"
gmt_output_file = "/content/corum_human.gmt"

# --- Processing Function ---
def create_corum_gmt(corum_file_path, output_gmt_path):
    """
    Parses the CORUM allComplexes.txt file and converts it to a GMT file.
    """
    try:
        corum_df = pd.read_csv(corum_file_path, sep='\t', header=0)
        corum_df = corum_df[corum_df['organism'] == 'Human']

        complex_count = 0
        with open(output_gmt_path, 'w') as gmt_file:
            for index, row in corum_df.iterrows():
                complex_name = row['complex_name']
                gene_list_str = row['subunits_gene_name']

                if isinstance(gene_list_str, str):
                    # *** THIS IS THE CORRECTED LINE ***
                    # It splits the genes, strips whitespace from each, and removes any that are empty.
                    genes = [gene.strip() for gene in gene_list_str.split(';') if gene.strip()]

                    # We only write the gene set if it has genes left after cleaning
                    if genes:
                        description = complex_name.replace('\t', ' ')
                        gmt_file.write(f"{complex_name}\t{description}\t" + "\t".join(genes) + "\n")
                        complex_count += 1

        print(f"Successfully created GMT file with {complex_count} human complexes at: {output_gmt_path}")
        print("\nFirst 3 lines of the .gmt file:")
        !head -n 3 {output_gmt_path}

    except FileNotFoundError:
        print(f"ERROR: The file was not found at '{corum_file_path}'. Please upload it.")
    except KeyError as e:
        print(f"ERROR: A required column was not found: {e}. The CORUM file format might have changed.")

# --- Run the function ---
create_corum_gmt(corum_input_file, gmt_output_file)

# Execute GSEA Pre-Ranked Analysis

In [ ]:
# --- Configuration ---
# These should match the output files from the previous steps.
rnk_file = "/content/corum_betweenness.rnk"
gmt_file = "/content/corum_human.gmt"
output_directory = "gsea_results_corum_centrality"

# --- Run GSEA Pre-Ranked ---
try:
    pre_res = gp.prerank(
        rnk=rnk_file,
        gene_sets=gmt_file,
        outdir=output_directory,
        min_size=3,      # Min size of a complex to be tested
        max_size=500,    # Max size of a complex to be tested
        permutation_num=1000,  # Number of permutations for significance testing
        format='png',    # Output plot format
        seed=42,         # For reproducible results
        verbose=True     # Show progress
    )

    print(f"\nAnalysis complete! Results are saved in the '{output_directory}' folder.")
    print("You can view and download the result files from the file browser on the left.")

except Exception as e:
    print(f"An error occurred during GSEA analysis: {e}")
    print("Please check that your .rnk and .gmt files were created successfully.")

In [ ]:
# @title Diagnose Gene ID Mismatch
#
# Description:
# This script checks the overlap between the gene identifiers in your .rnk file
# and your .gmt file. A low overlap is the most common reason for blank GSEA plots.

rnk_file = "/content/corum_betweenness.rnk"
gmt_file = "/content/corum_human.gmt"

try:
    # --- Read genes from the .rnk file ---
    # The first column contains the gene IDs.
    ranked_genes_df = pd.read_csv(rnk_file, sep='\t', header=None, usecols=[0])
    # Use .str.strip() to remove any accidental leading/trailing whitespace.
    ranked_genes = set(ranked_genes_df[0].str.strip())
    print(f"Found {len(ranked_genes)} unique genes in your ranked list (.rnk file).")
    # Print a few examples
    print(f"Examples from .rnk: {list(ranked_genes)[:5]}\n")


    # --- Read genes from the .gmt file ---
    gmt_genes = set()
    with open(gmt_file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            # Genes start from the 3rd column (index 2)
            for gene in parts[2:]:
                # Use .strip() here as well for safety
                gmt_genes.add(gene.strip())
    print(f"Found {len(gmt_genes)} unique genes across all CORUM complexes (.gmt file).")
    # Print a few examples
    print(f"Examples from .gmt: {list(gmt_genes)[:5]}\n")


    # --- Calculate and report the overlap ---
    overlapping_genes = ranked_genes.intersection(gmt_genes)

    print("--- DIAGNOSIS ---")
    print(f"Number of overlapping genes found in both files: {len(overlapping_genes)}")

    if len(ranked_genes) > 0:
      overlap_percentage = (len(overlapping_genes) / len(ranked_genes)) * 100
      print(f"Overlap percentage: {overlap_percentage:.2f}% of your ranked genes are in the CORUM GMT file.")

    if len(overlapping_genes) < 50:
        print("\nCONCLUSION: CRITICAL - Very low overlap detected!")
        print("This is almost certainly why your enrichment plots are blank.")
        print("Please check if the gene ID types are the same (e.g., both are HGNC Gene Symbols).")
        # Show which genes don't match
        if len(ranked_genes) > 0 and len(gmt_genes) > 0:
            print("\nExample of a gene in .rnk but NOT in .gmt:", list(ranked_genes - gmt_genes)[0])
            print("Example of a gene in .gmt but NOT in .rnk:", list(gmt_genes - ranked_genes)[0])

    else:
        print("\nCONCLUSION: Overlap looks reasonable. The issue might be more subtle.")

except FileNotFoundError as e:
    print(f"ERROR: A file was not found. Please ensure '{rnk_file}' and '{gmt_file}' exist. Error: {e}")

In [ ]:
# @title Check .rnk Score Column for Errors

rnk_file = "corum_betweenness.rnk"
df_check = pd.read_csv(rnk_file, sep='\t', header=None, names=['gene', 'score'])

# Check for non-numeric values
numeric_scores = pd.to_numeric(df_check['score'], errors='coerce')
nan_count = numeric_scores.isna().sum()

if nan_count > 0:
    print(f"WARNING: Found {nan_count} non-numeric or missing scores in your .rnk file!")
    print("This can also cause GSEA to fail. Please check your centrality calculation output.")
else:
    print("SUCCESS: The score column in your .rnk file appears to be clean (all numeric).")

# Graph vs. Hypergraph Comparison

In [ ]:
import pandas as pd

df_graph = pd.read_csv('gsea_results_GRAPH/gseapy.prerank.gene_sets.report.csv')
df_hypergraph = pd.read_csv('gsea_results_HYPERGRAPH/gseapy.prerank.gene_sets.report.csv')

# Merge the results based on the complex name ('Term')
df_comparison = pd.merge(
    df_graph[['Term', 'nes', 'fdr']],
    df_hypergraph[['Term', 'nes', 'fdr']],
    on='Term',
    suffixes=('_graph', '_hypergraph')
)

# Now you can easily find the differences
# Example: Find complexes significant in hypergraph but not graph (using FDR < 0.25 as cutoff)
significant_in_hypergraph_only = df_comparison[
    (df_comparison['fdr_hypergraph'] < 0.25) &
    (df_comparison['fdr_graph'] >= 0.25)
]

print(significant_in_hypergraph_only)